In [20]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
from scipy.optimize import milp, LinearConstraint
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
def open_input(day='10', suffix="input"):
    with open(f"{day}-{suffix}", "r") as file:
        lines = file.readlines()
    data_raw = [line.replace("\n", "") for line in lines]
    data_raw = "\n".join(data_raw)
    return data_raw

input_raw = open_input()
input_raw

'[.#.###] (0,1,2) (0,2,4,5) (3,5) (2,4,5) (0,1,3,4) (0,2,4) (5) {21,18,30,14,34,40}\n[.##.##.#] (0,4,5,6) (1,3,6) (0,1,2,3,4,5,6) (0,3,6,7) (1,2,3,4,6,7) (1,2,4,5) {206,71,52,235,56,42,239,196}\n[..##] (2,3) (0,3) (1,2) (0,2) {37,13,45,31}\n[##......] (3) (0,3,4,5,7) (2,3,7) (0,2,5) (1,2,4,5,7) (4,6,7) (0,1,2,4,5,6) (0,2,3,4,6) (3,4,6) {29,11,38,45,45,33,27,54}\n[#.....##] (5,7) (3,6) (0,2,3,4,6,7) (2,4) (0,6,7) (1,3,6,7) (1,2,5,6,7) {17,27,16,43,6,30,64,64}\n[##.#.#.#] (0,3,4,5,6) (3,5,6,7) (0,2,7) (0,1,6,7) (1,2,3,4,6,7) (0,3,4,7) (0,1,2,4,5,6,7) (0,1,3,5,7) (0,1,2,4,5) (0,3) {80,74,41,62,60,59,56,88}\n[#.#.] (0,2) (2,3) (0,3) (1,3) (3) (1,2) {26,7,25,27}\n[#..#####..] (4,5,7,8,9) (1,5,6,7,8,9) (1,7,9) (5) (0,2,3,6) (4,5,7) (0,3) (0,1,3,4,5,6) (2,6,8) (4,5) (6,8) (1,2,4,5,6,7,8,9) {17,30,9,17,49,84,47,62,63,47}\n[.#..###...] (0,2,3,4,6,7,8) (0,3,5,6,7,8) (1,5,7) (0,1,3,5,6,7) (0,2,3,4,6,8) (1,2,3,4,5,6,8,9) (2,6,7) (0,1,3,4,8) (7) (0,1,2,4,7,8) {48,25,54,58,48,22,64,47,53,11}\n[##..]

In [4]:
input_test_raw = open_input( suffix="test")
input_test_raw

'[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}\n[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}\n[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}'

In [5]:
def preprocess_data (data):
    # dtype='U10'
    

    maschines = []

    for row in data.split("\n"):
        indicator = re.findall(r'\[.*\]', row)[0].strip('[]')
        buttons = [[int (i) for i in b.strip("()").split(",")] for b in re.findall(r'\(.*?\)', row)]
        joltage = [int(i) for i in re.findall(r'\{.*\}', row)[0].strip('{}').split(",")  ]
        maschines.append([indicator, buttons, joltage])
    
    return maschines
test_data = preprocess_data(input_test_raw)
input_data = preprocess_data(input_raw)
display(test_data)

[['.##.', [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]], [3, 5, 4, 7]],
 ['...#.',
  [[0, 2, 3, 4], [2, 3], [0, 4], [0, 1, 2], [1, 2, 3, 4]],
  [7, 5, 12, 7, 2]],
 ['.###.#',
  [[0, 1, 2, 3, 4], [0, 3, 4], [0, 1, 2, 4, 5], [1, 2]],
  [10, 11, 11, 5, 10, 5]]]

In [42]:
def solution(data):
    solution = 0
    
    for maschine in data:
        indicator, buttons, joltage = maschine

        memory = {}
        configurations_seen = set()
        switch = {'#': '.', '.': '#'}

        def press_button(configuration, button):
            next_configuration = list(configuration)
            for slot in button:
                next_configuration[slot] = switch[next_configuration[slot]]
            
            return "".join(next_configuration)

        def min_presses(configuration, path):
            if configuration in path:
                return 9999999
            else:
                path.add(configuration)

            if configuration == indicator:
                return 0
            else:
                if configuration in memory.keys():
                    return memory[configuration]
                else:
                    sub_solutions = [min_presses(press_button(configuration, button), set(path))  for button in buttons]
                    sub_solution = 1 + min(sub_solutions)
                    memory[configuration] = sub_solution
                    # print(configuration, "sub solutions:", sub_solutions)
                    return sub_solution

        presses =  min_presses('.'*len(indicator), set())
        solution += presses

    
    return solution

sol_test = solution(test_data)
print("Test Solution:", sol_test)
sol_input = solution(input_data)
print("Input Solution:", sol_input)

Test Solution: 7
Input Solution: 481


In [ ]:


def solution2(data):
    solution = 0
    
    
    for maschine in tqdm(data):
        indicator, buttons, joltage = maschine

        memory = {}
        configurations_seen = set()
        switch = {'#': '.', '.': '#'}

        def press_button(configuration, button):
            next_configuration = configuration.copy()
            for slot in button:
                next_configuration[slot] += 1
            
            return next_configuration

        def list_to_str(config):
             return ",".join([str(i) for i in config])

        def min_presses(configuration, path):
            if any([config > target  for config, target in zip(configuration, joltage) ]):
                return 9999999
            # else:
            #     path.add(configuration)

            if all([config==target  for config, target in zip(configuration, joltage)]):
                return 0
            else:
                if list_to_str(configuration) in memory.keys():
                    return memory[list_to_str(configuration)]
                else:
                    sub_solutions = [min_presses(press_button(configuration, button), set())  for button in buttons]
                    sub_solution = 1 + min(sub_solutions)
                    memory[list_to_str(configuration)] = sub_solution
                    # print(configuration, "sub solutions:", sub_solutions)
                    return sub_solution

        presses =  min_presses([0 for i in joltage], set())
        solution += presses

    return solution


sol_test = solution2(test_data)
print("Test Solution:", sol_test)
sol_input = solution2(input_data)
print("Input Solution:", sol_input)

In [26]:


def solution2(data):
    solution = 0
    
    
    for maschine in tqdm(data):
        indicator, buttons, joltage = maschine
        # print(buttons, joltage)

        # x_1, ..., x_n one per button
        c = np.ones((len(buttons)))
        
        # bl=bu=joltages 
        bl = np.array(joltage)
        bu = np.array(joltage)

        # A: one row per battery, one column per button
        A=np.zeros((len(joltage), len(buttons)))
        for idx, button in enumerate(buttons):
            for slot in button:
                A[slot, idx] = 1

        constraints = LinearConstraint(A, bl, bu)
        # Integer variable; decision variable must be an integer within bounds.
        integrality = 1
        res = milp(c=c, constraints=constraints, integrality=integrality)
        
        solution += int(sum(res.x))

    return solution


sol_test = solution2(test_data)
print("Test Solution:", sol_test)
sol_input = solution2(input_data)
print("Input Solution:", sol_input)

100%|██████████| 3/3 [00:00<00:00, 520.13it/s]


Test Solution: 33


100%|██████████| 189/189 [00:00<00:00, 838.68it/s]

Input Solution: 20142
